# Build a quasar composite spectrum

# About

This notebook creates a composite quasar spectrum from a collection of quasar spectra.
The quasar spectra were retrieved from the *Sloan Digital Sky Survey* database DR 12 (http://skyserver.sdss.org/dr12/en/home.aspx) and stored in FITS files.

The notebook reads all FITS files in the given folder, corrects each individual spectrum for redshift (from observed frame to emitted frame), then re-bin-s the data in bins of a user-defined size. It then continues by calculating normalisation factors for each spectrum by the reciprocal of the division of the average spectral flux density in the overlapping region by the average spectral flux density of the adjacent (in the redshift range) spectrum in the same overlapping region.
Finally, all spectra are combined to one composite spectrum by calculating the *geometric* mean value of all values in a bin, for each bin.

## Credits

This notebook and associated Python modules were developed as part of a group project within the context of module S382 Astrophysics, lectured at the Open University in the UK (http://www.open.ac.uk/courses/modules/s382).

## Initialise

Auto reload libraries, use inline plots

In [1]:
%load_ext autoreload
#%matplotlib inline

Import the libraries

In [2]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as tic
import os

## Create a query for selecting 100 spectra evenly distributed between z = 0.1 and z = 6.9

In [ ]:
num_bins = 100
min_x = 0.1
max_x = 6.9
delta_x = (max_x-min_x)/num_bins

for i in range(1, num_bins):
    left_x = i*delta_x - 0.01
    right_x = i*delta_x + 0.01

    print("SELECT TOP 1 bestObjId, plate, mjd, fiberid")
    print("FROM SpecObj")
    print("WHERE class='QSO'")
    print("AND bestObjId>0")
    print("AND zWarning=0")
    print("AND z BETWEEN {0:.2f} AND {1:.2f}".format(left_x, right_x))
    print("UNION")

i = num_bins
left_x = i*delta_x - 0.01
right_x = i*delta_x + 0.01
print("SELECT TOP 1 bestObjId, plate, mjd, fiberid")
print("FROM SpecObj")
print("WHERE class='QSO'")
print("AND bestObjId>0")
print("AND zWarning=0")
print("AND z BETWEEN {0:.2f} AND {1:.2f}".format(left_x, right_x))

## A query for selecting ~2000 quasars from the SDSS with latitude > 85 degrees.

In [ ]:
# SELECT t1.bestobjid, t1.z, t1.plate, t1.mjd, t1.fiberid, t2.b
# FROM specobj t1, photoobj t2
# WHERE t1.bestobjid = t2.objid
# AND t1.zwarning=0
# AND t1.class='QSO'
# AND t1.subclass='BROADLINE'
# AND t1.bestobjid>0
# AND t1.survey = 'boss'
# AND t2.b > 85

## Query the database

Copy & Paste the generated SQL query in the *Skyserver SQL Search* box at http://skyserver.sdss.org/dr12/en/tools/search/sql.aspx.

Next, download all FITS files - most easily done using the *Bulk Optical Spectra Search* at http://mirror.sdss3.org/bulkSpectra.

## Create a composite spectrum using homebrewn routines

In [4]:
# %autoreload
import hkCompositeSpectrum as hsc

#
# Create composite spectrum
#
binSize = 4.
foldername = "../../Downloads/FITS2/"

# Store the names of fits-files
filenameList = []
for filename in os.listdir(foldername):
    if (filename.endswith(".fits") or filename.endswith(".FITS")):
        filenameList.append(foldername + filename)

# Create the composite spectrum
compositeSpectrum = hsc.CompositeSpectrum(filenameList, binSize = 4., method="GM")

# Store in CSV file
filename = './FITS/hkCompositeSpectrum.csv'
print("Writing composite spectrum to " + filename)
compositeSpectrum.to_csv(filename, index=False)
print("Done...")

Reading 1990 FITS files...
Processing: ../../Downloads/FITS2/spec-5984-56337-0538.fits...
Processing: ../../Downloads/FITS2/spec-6484-56358-0754.fits...
Processing: ../../Downloads/FITS2/spec-5983-56310-0972.fits...
Processing: ../../Downloads/FITS2/spec-6483-56341-0742.fits...
Processing: ../../Downloads/FITS2/spec-5989-56312-0510.fits...
Processing: ../../Downloads/FITS2/spec-5988-56072-0556.fits...
Processing: ../../Downloads/FITS2/spec-5984-56337-0658.fits...
Processing: ../../Downloads/FITS2/spec-6484-56358-0886.fits...
Processing: ../../Downloads/FITS2/spec-5991-56076-0498.fits...
Processing: ../../Downloads/FITS2/spec-5981-56340-0273.fits...
Processing: ../../Downloads/FITS2/spec-5968-56356-0388.fits...
Processing: ../../Downloads/FITS2/spec-5986-56328-0410.fits...
Processing: ../../Downloads/FITS2/spec-6483-56341-0266.fits...
Processing: ../../Downloads/FITS2/spec-5981-56340-0876.fits...
Processing: ../../Downloads/FITS2/spec-5989-56312-0022.fits...
Processing: ../../Downloads/

## Plot the composite spectrum, including common emission lines

In [6]:
%autoreload
import hkSpectrumLines as hsl

# Import data
compositeSpectrum = pd.read_csv('./FITS/hkCompositeSpectrum.csv', skiprows=0)

#
# Plot the spectrum
#

# Create a figure with axes
fig = plt.figure()
ax0 = fig.add_subplot(1, 1, 1)

# Plot the composite spectrum
ax0.plot(compositeSpectrum['wavelength'], compositeSpectrum['mean_f_lambda'], linewidth=1)

# Plot spectrum lines
hsl.PlotSpectrumLines(ax0,[r'Ly$\alpha$', 'C IV', 'C III]', 'Mg II', r'H$\beta$', '[O III]', r'H$\alpha$'], 14)

# Set limits for axes
#ax0.set_xlim(1000,8500)
#ax0.set_ylim(0, 1800)

# Create labels for axes
ax0.set_xlabel('Wavelength $\lambda / \AA$', fontsize='large')
ax0.set_ylabel('Spectral flux density $f_{\lambda} \, / $ Arbitrary units', fontsize='large')

# Create a title
ax0.set_title('Spectral flux densities for composite spectrum of quasar', fontsize='xx-large')

# Display a grid
ax0.grid(True)

# Set the tick marks
xMajorLocator = tic.MultipleLocator(500)
xMinorLocator = tic.AutoMinorLocator(5)
ax0.xaxis.set_major_locator(xMajorLocator)
ax0.xaxis.set_minor_locator(xMinorLocator)

yMinorLocator = tic.AutoMinorLocator(5)
ax0.yaxis.set_minor_locator(yMinorLocator)

ax0.tick_params(which = 'both', direction = 'in')

# Display the plot
plt.show()

## Create a signal-to-noise plot

In [7]:
# Read the data file into a dataframe using pandas
qsoData = pd.read_csv('./FITS/hkCompositeSpectrum.csv', skiprows=0)

# Calculate signal-to-noise
qsoData['SNR1'] = qsoData['mean_f_lambda']/qsoData['noise_f_lambda']
qsoData['SNR2'] = qsoData['mean_f_lambda']/qsoData['unc_f_lambda']

#
# Plot the spectrum
#

# Create a figure with axes
fig = plt.figure()

# Plot the composite spectrum
ax1 = fig.add_subplot(1, 1, 1)
ax1.plot(qsoData['wavelength'], qsoData['SNR1'], linewidth=1.5, color='blue', label='S/N using noise_f_lambda')
ax1.plot(qsoData['wavelength'], qsoData['SNR2'], linewidth=1.5, color='green', label='S/N using unc_f_lambda')

ax1.set_xlabel('Emitted wavelength $\lambda / \AA$', fontsize='x-large', fontweight='normal')
ax1.set_ylabel('S/N', fontsize='x-large', fontweight='normal')

# Set axis limits and plot grid
ax1.set_xlim(1000,7000)
ax1.set_ylim(0, 500)
ax1.grid(True)

# Create a title
ax1.set_title('Signal-to-noise ratio for the Composite Quasar Spectrum', fontsize='xx-large', fontweight='bold')

# Set the tick marks
xMajorLocator = tic.MultipleLocator(500)
xMinorLocator = tic.AutoMinorLocator(5)
ax1.xaxis.set_major_locator(xMajorLocator)
ax1.xaxis.set_minor_locator(xMinorLocator)

yMinorLocator = tic.AutoMinorLocator(5)
ax1.yaxis.set_minor_locator(yMinorLocator)

ax1.tick_params(which = 'both', direction = 'in')

# Now add the legend with some customizations.
legend = ax1.legend()

# Display the plot
plt.show()

## Create a continuum

In [8]:
%autoreload
import hkCompositeSpectrum as hcs
import hkSpectrumLines as hsl

filename = './FITS/hkCompositeSpectrum.csv'

# Import data
qso = pd.read_csv(filename, skiprows=0)
qso['continuum'], tempDf = hcs.CreateContinuum(qso, [1080, 5200])
qso.to_csv(filename, index=False)

# Create a figure with axes
fig = plt.figure()

ax0 = fig.add_subplot(1, 1, 1)

# Plot the rebinned data
ax0.plot(qso['wavelength'], qso['mean_f_lambda'], linewidth=1, label='Composite spectrum')
ax0.plot(tempDf['wavelength'], tempDf['mean_f_lambda'], linewidth=1, label='Composite spectrum after emission line removal')
ax0.plot(qso['wavelength'], qso['continuum'], linewidth=1, label='Final power law functions describing the continuum')

# Plot spectrum lines
#hsl.PlotSpectrumLines(ax0,[r'Ly$\alpha$', 'C IV', 'C III]', 'Mg II', r'H$\beta$', '[O III]', r'H$\alpha$'], 170)

# Create labels for axes
ax0.set_xlabel('Emitted wavelength $\lambda / \AA$', fontsize='xx-large', fontweight='normal')
ax0.set_ylabel('Spectral flux density $f_{\lambda} \, / $ Arbitrary units', fontsize='xx-large', fontweight='normal')

# Create a title
ax0.set_title('Spectral flux density for the composite quasar spectrum', fontsize='xx-large', fontweight='bold')

# Set limits for axes
#ax0.set_xlim(1450,7000)
#ax0.set_ylim(0, 70)

# Display a grid
ax0.grid(True)

idx = qso[qso['wavelength']==2200].index
x = qso['wavelength'].iloc[idx]
y = qso['continuum'].iloc[idx]
text = r'$F_{\lambda} \propto \lambda^{-1.02}$'
plt.annotate(text, xy=(x, y), arrowprops=dict(arrowstyle='->', connectionstyle='arc3, rad=+0.3'), xytext=(+25, +100), xycoords = 'data', textcoords='offset points', fontsize='x-large')

idx = qso[qso['wavelength']==6560].index
x = qso['wavelength'].iloc[idx]
y = qso['continuum'].iloc[idx]
text = r'$F_{\lambda} \propto \lambda^{+1.23}$'
plt.annotate(text, xy=(x, y), arrowprops=dict(arrowstyle='->', connectionstyle='arc3, rad=-0.3'), xytext=(-125, +100), xycoords = 'data', textcoords='offset points', fontsize='x-large')

# Now add the legend with some customizations.
legend = ax0.legend()

# The frame is matplotlib.patches.Rectangle instance surrounding the legend.
#frame = legend.get_frame()
#frame.set_facecolor('0.90')

# Display the plot
plt.show()

slope = 1.35; intercept = -8.92; std_err = 0.141
slope = 0.015; intercept = 0.189; std_err = 0.0175
slope = 0.738; intercept = -5.88; std_err = 0.0564
new breakpoint at 907 A
new breakpoint at 4.37E+03 A
slope = 0.665; intercept = -4.38; std_err = 0.105
slope = 0.116; intercept = -0.696; std_err = 0.00959
slope = 0.841; intercept = -6.81; std_err = 0.0511
new breakpoint at 818 A
new breakpoint at 4.63E+03 A


Exception in Tkinter callback
Traceback (most recent call last):
  File "/usr/lib64/python3.5/tkinter/__init__.py", line 1549, in __call__
    return self.func(*args)
  File "/usr/lib64/python3.5/site-packages/matplotlib/backends/backend_tkagg.py", line 283, in resize
    self.show()
  File "/usr/lib64/python3.5/site-packages/matplotlib/backends/backend_tkagg.py", line 354, in draw
    FigureCanvasAgg.draw(self)
  File "/usr/lib64/python3.5/site-packages/matplotlib/backends/backend_agg.py", line 474, in draw
    self.figure.draw(self.renderer)
  File "/usr/lib64/python3.5/site-packages/matplotlib/artist.py", line 61, in draw_wrapper
    draw(artist, renderer, *args, **kwargs)
  File "/usr/lib64/python3.5/site-packages/matplotlib/figure.py", line 1159, in draw
    func(*args)
  File "/usr/lib64/python3.5/site-packages/matplotlib/artist.py", line 61, in draw_wrapper
    draw(artist, renderer, *args, **kwargs)
  File "/usr/lib64/python3.5/site-packages/matplotlib/axes/_base.py", line 2324

## Calculate the FWHM of common emission lines

In [ ]:
%autoreload
import hkCompositeSpectrum as hcs

#
# Calculate the velocity dispersions
#

# Import data
qso = pd.read_csv('./FITS/hkCompositeSpectrum.csv', skiprows=0)

# Calculate the Full-Width Half-Maximum
FwhmDf = hcs.CalculateFwhm1(qso, [r'Ly$\alpha$', 'C IV', 'C III]', 'Mg II', r'H$\beta$', '[O III]', r'H$\alpha$'])

for idx in FwhmDf.index:
    print("Line {0} is located at {1:4.2f}Å, has width {2:4.2f}Å and velocity dispersion {3:.3G} km s^-1".format(FwhmDf['Line'].iloc[idx], FwhmDf['Wavelength'].iloc[idx], FwhmDf['Width'].iloc[idx], FwhmDf['Width'].iloc[idx]/FwhmDf['Wavelength'].iloc[idx]*2.998E5))

## Create a log-log plot of the composite spectrum

In [ ]:
#
# Plot the spectrum in loglog scale
#

# Import data
qsoA = pd.read_csv('./FITS/hkCompositeSpectrum.csv', skiprows=0)

# Create a theoretical spectrum
A1     = 2.70E6
alpha1 = -1.56
A2     = 9.17E-2
alpha2 = 0.45
bp1     = 5200

qsoA['theoretical'] = (qsoA['wavelength'] <= bp1)*A1*qsoA['wavelength']**alpha1 + (qsoA['wavelength'] > bp1)*A2*qsoA['wavelength']**alpha2
qsoA['theoretical'][qsoA['wavelength']<1080] = 0

# Create a best fit for continuum
A3     = np.exp(8.27)
alpha3 = -1.02
A4     = np.exp(-10.8)
alpha4 = +1.23
bp2     = 4750 #4.89E3

qsoA['continuum'] = (qsoA['wavelength'] <= bp2)*A3*qsoA['wavelength']**alpha3 + (qsoA['wavelength'] > bp2)*A4*qsoA['wavelength']**alpha4
qsoA['continuum'][qsoA['wavelength']<1080] = 0

# Create a figure with axes
fig = plt.figure()
ax0 = fig.add_subplot(1, 1, 1)

# Plot the composite spectrum
ax0.loglog(qsoA['wavelength'], qsoA['mean_f_lambda'], linewidth=1.5, color='lightblue', label='Composite spectrum')
ax0.loglog(qsoA['wavelength'], qsoA['continuum'], linewidth=2, color='red', label='Best fit trend lines')
#ax0.loglog(qsoA['wavelength'], qsoA['theoretical'], linewidth=2, linestyle='dashed', color='green', label='Trend lines used for normalisation (source: Vanden Berk et al.)')

# Add annotations for continuum
#idx = qsoA.idxmin
idx = qsoA[qsoA['wavelength']==1080].index
x = qsoA['wavelength'].iloc[idx]
y = qsoA['continuum'].iloc[idx]
text = r'$\alpha_\lambda = {0:.2f}$'.format(alpha3)
plt.annotate(text, xy=(x, y), arrowprops=dict(arrowstyle='->', connectionstyle='arc3, rad=-0.3'), xytext=(+25, -100), xycoords = 'data', textcoords='offset points', fontsize='x-large')

#idx = qsoA.idxmax
idx = qsoA[qsoA['wavelength']==6548].index
x = qsoA['wavelength'].iloc[idx]
y = qsoA['continuum'].iloc[idx]
text = r'$\alpha_\lambda = {0:.2f}$'.format(alpha4)
plt.annotate(text, xy=(x, y), arrowprops=dict(arrowstyle='->', connectionstyle='arc3, rad=-0.3'), xytext=(-75, +100), xycoords = 'data', textcoords='offset points', fontsize='x-large')

# Add annotations for theoretical
#idx = qsoA.idxmin
idx = qsoA[qsoA['wavelength']==1080].index
x = qsoA['wavelength'].iloc[idx]
y = qsoA['theoretical'].iloc[idx]
text = r'$\alpha_\lambda = {0:.2f}$'.format(alpha1)
plt.annotate(text, xy=(x, y), arrowprops=dict(arrowstyle='->', connectionstyle='arc3, rad=0.3'), xytext=(+25, +100), xycoords = 'data', textcoords='offset points', fontsize='x-large')

#idx = qsoA.idxmax
idx = qsoA[qsoA['wavelength']==6548].index
x = qsoA['wavelength'].iloc[idx]
y = qsoA['theoretical'].iloc[idx]
text = r'$\alpha_\lambda = {0:.2f}$'.format(alpha2)
plt.annotate(text, xy=(x, y), arrowprops=dict(arrowstyle='->', connectionstyle='arc3, rad=0.3'), xytext=(-75, -100), xycoords = 'data', textcoords='offset points', fontsize='x-large')

# Set limits for axes
#ax0.set_xlim(1000,7000)
#ax0.set_ylim(0, 1800)

# Create labels for axes
ax0.set_xlabel('Emitted wavelength $\lambda / \AA$', fontsize='xx-large', fontweight='normal')
ax0.set_ylabel('Spectral flux density $f_{\lambda} \, / $ Arbitrary units', fontsize='xx-large', fontweight='normal')

# Create a title
ax0.set_title('Spectral flux densities for the composite quasar spectrum with trend lines', fontsize='xx-large', fontweight='bold')

# Display a grid
ax0.grid(True)

# Set the tick marks
#xMajorLocator = tic.MultipleLocator(500)
#xMinorLocator = tic.AutoMinorLocator(5)
#ax0.xaxis.set_major_locator(xMajorLocator)
#ax0.xaxis.set_minor_locator(xMinorLocator)

#yMinorLocator = tic.AutoMinorLocator(5)
#ax0.yaxis.set_minor_locator(yMinorLocator)

ax0.set_xticks([1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000])
ax0.set_yticks([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100])
ax0.tick_params(which = 'both', direction = 'in')

# Now add the legend with some customizations.
legend = ax0.legend()

# The frame is matplotlib.patches.Rectangle instance surrounding the legend.
frame = legend.get_frame()
frame.set_facecolor('0.90')

# Display the plot
plt.show()